In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import numpy as np
from PIL import Image
import base64
from io import BytesIO

import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
sio = socketio.Server()
app = Flask(__name__)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [3]:
model = load_model('mymodel.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 56, 76, 32)        18464     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 26, 36, 16)        4624      
_______________________________________

In [ ]:
def process_image(img):
    #160x320x3 -> 60x80x3
    return img[10:130:2, ::4, :]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        decoded = base64.b64decode(image_str)
        #print(type(decoded))
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)        

        steering_angle = float(model.predict(img_batch))

        #steering_angle = 0.2 # -1..1
        throttle = 0.15 # 0..1
        if speed < 15:
            throttle = 0.9
        if speed > 17:
            throttle = -0.1
            

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(7173) wsgi starting up on http://0.0.0.0:4567
(7173) accepted ('127.0.0.1', 42867)
(7173) accepted ('127.0.0.1', 35437)
127.0.0.1 - - [19/May/2019 11:38:01] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 153.158516
127.0.0.1 - - [19/May/2019 11:38:19] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 72.466907
(7173) accepted ('127.0.0.1', 39273)
(7173) accepted ('127.0.0.1', 44987)
127.0.0.1 - - [19/May/2019 11:39:53] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 62.879822
